# Satellite albedo

## Import packages

In [ ]:
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download, plot, utils

## Define Parameters

In [ ]:
year_start = 2014
year_stop = 2015

area = [15, 75, 10, 80]
subset = [13.2, 77.4, 12.8, 77.8]

## Define requests

In [ ]:
# Albedo
collection_id_albedo = "satellite-albedo"
request_albedo = {
    "format": "zip",
    "variable": "albb_bh",
    "satellite": "proba",
    "sensor": "vgt",
    "product_version": "v2",
    "horizontal_resolution": "1km",
    "year": [str(year) for year in range(year_start, year_stop + 1)],
    "month": [f"{month:02d}" for month in range(1, 13)],
    "nominal_day": "10",
    "area": area,
}

# 2m temperature
collection_id_t2m = "reanalysis-era5-land-monthly-means"
request_t2m = {
    "format": "grib",
    "product_type": "monthly_averaged_reanalysis",
    "variable": "2m_temperature",
    "time": "00:00",
    "year": [str(year) for year in range(year_start, year_stop + 1)],
    "month": [f"{month:02d}" for month in range(1, 13)],
    "area": subset,
}

## Download and transform data

In [ ]:
# Albedo maps
ds_albedo_maps = download.download_and_transform(
    collection_id_albedo,
    request_albedo,
    transform_func=diagnostics.time_weighted_mean,
    transform_chunks=False,
    split_all=True,
    drop_variables=["crs"],
)

# Albedo timeseries
ds_albedo_timeseries = download.download_and_transform(
    collection_id_albedo,
    request_albedo,
    transform_func=diagnostics.spatial_weighted_std,
    split_all=True,
    drop_variables=["crs"],
)

# 2m temperature maps
ds_t2m_maps = download.download_and_transform(
    collection_id_t2m,
    request_t2m,
    transform_func=diagnostics.time_weighted_mean,
    transform_chunks=False,
    chunks={"year": 1},
)
with xr.set_options(keep_attrs=True):
    ds_t2m_maps["t2m"] -= 273.15
ds_t2m_maps["t2m"].attrs["units"] = "°C"

## Plot albedo maps

In [ ]:
da_albedo = ds_albedo_maps["AL_BH_BB"]
colors = ["blue", "skyblue", "cyan", "lightgreen", "yellow", "orange", "red", "maroon"]
custom_cmap = mcolors.LinearSegmentedColormap.from_list("CustomColorMap", colors)

plot.projected_map(da_albedo, cmap=custom_cmap)
plt.show()

lon_slice = slice(subset[1], subset[3])
lat_slice = slice(subset[2], subset[0])
da_albedo_subset = utils.regionalise(
    da_albedo, lon_slice=lon_slice, lat_slice=lat_slice
)
_ = plot.projected_map(da_albedo_subset, cmap=custom_cmap)

## Plot albedo timeseries

In [ ]:
ds_albedo_timeseries["AL_BH_BB"].plot(marker="o", linestyle="-", color="b")
plt.grid()

## Plot 2m temperature maps

In [ ]:
da_t2m = ds_t2m_maps["t2m"]
plot.projected_map(da_t2m, cmap="YlOrRd")
plt.title("original")
plt.show()

da_t2m_regrid = diagnostics.regrid(
    da_t2m, grid_out=da_albedo_subset, method="nearest_s2d"
)
plot.projected_map(da_t2m_regrid, cmap="YlOrRd")
_ = plt.title("regridded")